In [1]:
import os
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import lineid_plot
from ramandecompy import spectrafit
from ramandecompy import peakidentify
from ramandecompy import dataprep
from ramandecompy import datavis


In [2]:
dataprep.new_hdf5('Experimental_Mixture_Spectra_300C_25s')

OSError: Unable to create file (unable to open file: name = 'Experimental_Mixture_Spectra_300C_25s.hdf5', errno = 17, error message = 'File exists', flags = 15, o_flags = 502)

In [ ]:
hdf5_filename = 'Experimental_Mixture_Spectra_300C_25s.hdf5'
key = '300C/25s'

In [ ]:
dataprep.add_experiment(hdf5_filename, '../ramandecompy/tests/test_files/FA_3.6wt%_300C_25s.csv')
# dataprep.add_experiment(hdf5_filename, '../ramandecompy/tests/test_files/FA_3.6wt%_300C_35s.csv')
# dataprep.add_experiment(hdf5_filename, '../ramandecompy/tests/test_files/FA_3.6wt%_300C_45s.csv')
# dataprep.add_experiment(hdf5_filename, '../ramandecompy/tests/test_files/FA_3.6wt%_300C_55s.csv')
# dataprep.add_experiment(hdf5_filename, '../ramandecompy/tests/test_files/FA_3.6wt%_300C_65s.csv')

In [ ]:
add_list = [1270, 1350, 1385]#, 1640]
# add_list = None
drop_list = ['Peak_01']#, 'Peak_02']
# drop_list = None

dataprep.adjust_peaks(hdf5_filename, key, add_list, drop_list, plot_fits=True)

In [ ]:
hdf5_calfilename = 'peakidentify_calibration_file.hdf5' #update to hdf5_calfilename
hdf5_expfilename = 'Experimental_Mixture_Spectra_300C_25s.hdf5'
expkey = '300C/25s'
df = peakidentify.peak_assignment(hdf5_expfilename, expkey, hdf5_calfilename, 20,plot=True)
plt.savefig('Adjusted_peakidentification_labeled', dpi=300, bbox_inches='tight')

In [ ]:
hdf5_calfilename = 'peakidentify_calibration_file.hdf5'
key1 = 'CarbonMonoxide'
key2 = 'Hydrogen'
key3 = 'CO2'
key4 = 'sapphire'
key5 = 'H2O'
# key6 = 'Propane'
# key7 = 'Ethane'
# key8 = 'Acetaldehyde'
key9 = 'FormicAcid'
hdf5_expfilename = 'Experimental_Mixture_Spectra_300C_25s.hdf5'
expkey = '300C/25s'
# open .hdf5
calhdf5 = h5py.File(hdf5_calfilename, 'r+')
exphdf5 = h5py.File(hdf5_expfilename, 'r+')
# extract spectra data
residuals = np.asarray(list(exphdf5['{}/residuals'.format(expkey)]))
unknown_x = list(exphdf5['{}/wavenumber'.format(expkey)])
unknown_y = list(exphdf5['{}/counts'.format(expkey)])
# extract fitted peak center values
co_peaks = []
H_peaks = []
CO2_peaks = []
sapphire_peaks = []
H2O_peaks = []
prop_peaks = []
ethane_peaks = []
alde_peaks = []
formic_peaks = []
unknown_peaks = []
for _,peak in enumerate(list(calhdf5[key1])[:-3]):
    co_peaks.append(list(calhdf5['{}/{}'.format(key1, peak)])[0][2])

for _,peak in enumerate(list(calhdf5[key2])[:-3]):
    H_peaks.append(list(calhdf5['{}/{}'.format(key2, peak)])[0][2])
for _,peak in enumerate(list(calhdf5[key3])[:-3]):
    CO2_peaks.append(list(calhdf5['{}/{}'.format(key3, peak)])[0][2])
for _,peak in enumerate(list(calhdf5[key4])[:-3]):
    sapphire_peaks.append(list(calhdf5['{}/{}'.format(key4, peak)])[0][2])
for _,peak in enumerate(list(calhdf5[key5])[:-3]):
    H2O_peaks.append(list(calhdf5['{}/{}'.format(key5, peak)])[0][2])
# for _,peak in enumerate(list(calhdf5[key6])[:-3]):
#     prop_peaks.append(list(calhdf5['{}/{}'.format(key6, peak)])[0][2])
# for _,peak in enumerate(list(calhdf5[key7])[:-3]):
#     ethane_peaks.append(list(calhdf5['{}/{}'.format(key7, peak)])[0][2])
# for _,peak in enumerate(list(calhdf5[key8])[:-3]):
#     alde_peaks.append(list(calhdf5['{}/{}'.format(key8, peak)])[0][2])
for _,peak in enumerate(list(calhdf5[key9])[:-3]):
    formic_peaks.append(list(calhdf5['{}/{}'.format(key9, peak)])[0][2])
unknown_peaks = []
for i, peak in enumerate(list(exphdf5['{}'.format(expkey)])[:-3]):
    try:
        if i < 9:
            unknown_peaks.append(list(exphdf5['{}/Peak_0{}'.format(expkey,
                                                                  i+1)])[0][2])
        else:
            unknown_peaks.append(list(exphdf5['{}/Peak_{}'.format(expkey,
                                                                 i+1)])[0][2])
    except Exception as e:
        #Normal peakassignment
        print("""Function did not receive normal peak.
        The function continued to look for an adjusted peak.""")
        if i < 9:
            print(peak)
            unknown_peaks.append(list(exphdf5['{}/Peak_0{}*'.format(expkey,
                                                                   i+1)])[0][2])
        else:
            unknown_peaks.append(list(exphdf5['{}/Peak_{}*'.format(expkey,
                                                                  i+1)])[0][2])
        print('Peak_{}*'.format(i+1))
    else:
        pass
unknown_x = np.asarray(unknown_x)
unknown_y = np.asarray(unknown_y)
known_compound_list = list(calhdf5.keys())
precision = 10
# known_peaks_listtest = [alde_peaks,co_peaks, CO2_peaks,ethane_peaks,formic_peaks, H_peaks, H2O_peaks, sapphire_peaks,prop_peaks]
known_peaks_listtest = [co_peaks, CO2_peaks,formic_peaks, H_peaks, H2O_peaks, sapphire_peaks]
known_peakstest = []
association_matrixtest = []
# for i, _ in enumerate(known_compound_list):
#     for _,peak in enumerate(list(hdf5[key])[:-3]):
#         known_peakstest.append(known_peaks_listtest[i])
#         #print(type(known_peaks))
#         association_matrixtest.append(compare_unknown_to_known(
#             unknown_peakstest, known_peakstest[i], precision,
#             hdf5_expfilename, expkey))

        
#OK, next identify all of the peaks present in the known compound set.
    #For efficiency, we'll also compare them against the unknown in the same for loop.
known_peaks = []
known_peaks_list = []
num_peaks_list = []
association_matrix = []
split__index_list = []
for i, _ in enumerate(known_compound_list):
    print("The peaks that we found for "
      + str(known_compound_list[i]) + " are: ")
    num_peaks_list.append(len(list(calhdf5[known_compound_list[i]])[:-3]))
    split__index_list.append(sum(num_peaks_list))
    for j,peak in enumerate(list(calhdf5[known_compound_list[i]])[:-3]):
        print(list(calhdf5['{}/{}'.format(known_compound_list[i], peak)])[0][2])
        # Need to separate known peaks to make a list of two separate lists
        # to perform custom list split using list comprehension + zip() and split_index_list
        known_peaks_list.append(list(calhdf5['{}/{}'.format(known_compound_list[i], peak)])[0][2])
        result = [known_peaks_list[i : j] for i, j in zip([0] + split__index_list, split__index_list + [None])] 
    known_peaks.append(result)
    association_matrix.append(peakidentify.compare_unknown_to_known(
        unknown_peaks, known_peaks[i][i], precision))        
       

unknown_peak_assignments = peakidentify.peak_position_comparisons(
    unknown_peaks,
    known_peaks,
    association_matrix,
    hdf5_calfilename)
# open .hdf5
residuals = np.asarray(list(exphdf5['{}/residuals'.format(key)]))
#Extract keys from files
known_compound_list = list(calhdf5.keys())

# extract spectra data
x_data =  list(exphdf5['{}/wavenumber'.format(expkey)])
y_data = list(exphdf5['{}/counts'.format(expkey)])
peak_labels=[]
for i, _ in enumerate(unknown_peak_assignments):  
        peak_labels.append(str(unknown_peak_assignments[i]))
frames = []
for j, peak in enumerate(list(exphdf5['{}'.format(expkey)])[:-3]):
    frames.append(peakidentify.add_label(hdf5_expfilename, expkey, peak, peak_labels[j]))

df = pd.concat(frames,axis=1, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True,sort=True)
df =df.T

peak_labels = []
for i, _ in enumerate(unknown_peak_assignments):
    peak_labels.append(str(unknown_peak_assignments[i]))
#         print(peak_labels)
# plot spectra and peak labels
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True,
                           gridspec_kw={'height_ratios': [3, 1]},
                           figsize=(15, 6), dpi=300)
# plot data
ax1.plot(x_data, y_data, color='blue')
ax2.plot(x_data, residuals, color='teal')
lineid_plot.plot_line_ids(x_data, y_data, unknown_peaks,
                      peak_labels, box_axes_space=0.30,
                      plot_kwargs={'linewidth':1},
                      max_iter=75, ax=ax1)
#     fig.set_size_inches(15,5)
# lock the scale so that additional plots do not warp the labels
ax1.set_autoscale_on(False)
# Titles and labels
ax2.set_xlabel('Wavenumber ($cm^{-1}$)', fontsize=14)
ax1.set_xlim(min(x_data), max(x_data))
ax1.set_ylabel('Intensity (arb. units)', fontsize=14, labelpad=20)
ax2.set_ylabel('Residuals', fontsize=14, labelpad=12)
# scale residuals plot symmetrically about zero
ylim = max(abs(min(residuals)), abs(max(residuals)))
ax2.set_ylim(-ylim, ylim)
# add grid lines to residual plot
ax2.grid(which='major', axis='y', linestyle='-')
# force tick labels for top plot
ax1.tick_params(axis='both', which='both', labelsize=10, labelbottom=True)
# add title
ax1.set_title('{} spectra from {}'.format(key, 'Experimental_Mixture_Spectra_300C_25s'),
          fontsize=18, pad=250)
plt.show()
plt.savefig('Adjusted_peakidentification_labelednew', dpi=300, bbox_inches='tight')
#                                             association_matrix,
#                                             hdf5_calfilename)
# print(percentages)
calhdf5.close()

In [ ]:
# plot spectra and peak labels
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True,
                           gridspec_kw={'height_ratios': [3, 1]},
                           figsize=(15, 6), dpi=300)
# plot data
ax1.plot(x_data, y_data, color='blue')
ax2.plot(x_data, residuals, color='teal')
lineid_plot.plot_line_ids(x_data, y_data, unknown_peaks,
                      peak_labels, box_axes_space=0.30,
                      plot_kwargs={'linewidth':1},
                      max_iter=75, ax=ax1)
#     fig.set_size_inches(15,5)
# lock the scale so that additional plots do not warp the labels
ax1.set_autoscale_on(False)
# Titles and labels
ax2.set_xlabel('Wavenumber ($cm^{-1}$)', fontsize=14)
ax1.set_xlim(min(x_data), max(x_data))
ax1.set_ylabel('Intensity (arb. units)', fontsize=14, labelpad=20)
ax2.set_ylabel('Residuals', fontsize=14, labelpad=12)
# scale residuals plot symmetrically about zero
ylim = max(abs(min(residuals)), abs(max(residuals)))
ax2.set_ylim(-ylim, ylim)
# add grid lines to residual plot
ax2.grid(which='major', axis='y', linestyle='-')
# force tick labels for top plot
ax1.tick_params(axis='both', which='both', labelsize=10, labelbottom=True)
# add title
ax1.set_title('{} spectra from {}'.format(key, 'Experimental_Mixture_Spectra_300C_25s'),
          fontsize=18, pad=250)
plt.show()
plt.savefig('Adjusted_peakidentification_labelednew3', dpi=300, bbox_inches='tight')

In [ ]:
peakidentify.plotting_peak_assignments(unknown_x, unknown_y, unknown_peaks, unknown_peak_assignments, hdf5_expfilename, hdf5_calfilename, expkey, peak_labels, exportlabelinput = False )
